In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

In [3]:
from ml.preprocess import data
from utils import dasker, helper

In [4]:

from torch.utils.data import random_split, DataLoader, Dataset
from dask_ml.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np


In [5]:
N_SPLITS = 3

In [6]:


X_train_orig, y = data.get_dataset(f_name = '20220319_covid_merge_processed.csv' , sep = ',')
# X_tilda = X.drop(brisk_xgboost.DROP_LIST, axis = 1)
X = X_train_orig.drop(['location'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)



In [7]:
DROP_LIST = ['location']

def get_dataset():
    df = pd.read_csv('/mnt/c/Users/rwmas/GitHub/xai/xai_api/app/test/data/20220319_covid_merge_processed.csv', sep=",")

    X = df[df.columns[df.columns!='y']]

    X = X.drop(DROP_LIST, axis = 1)

    y = df[df.columns[df.columns=='y']]

    return X, y



def get_split_dataset():
    X, y = get_dataset()
    return train_test_split(X, y, random_state=0, test_size=0.33)


In [8]:
X_train, X_test, y_train, y_test,  = get_split_dataset()

In [9]:
# y_train.values.reshape(y_train.shape[0], )

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold, KFold

In [11]:
params = {
    'n_estimators': 100,
    'criterion' : 'squared_error',
    'max_depth': 2
}

In [22]:
regr = RandomForestRegressor(max_depth=params['max_depth'], random_state=0)
regr.fit(X_train, y_train)

RandomForestRegressor(max_depth=2, random_state=0)

In [51]:
regr.score(X_train, y_train)

0.7388743460063523

In [52]:
regr.score(X_test, y_test)

0.4862370637506407

In [24]:
pred_regr = regr.predict(X_test)

In [24]:
np.round(mean_squared_error(pred_regr, y_test), 2)

NameError: name 'pred_regr' is not defined

In [40]:
from sklearn.model_selection import cross_val_score, cross_validate
# from sklearn.model_selection.


def objective(trial):

    # X, y  = get_dataset()

    X_train, X_test, y_train, y_test,  = get_split_dataset()

    y_train = y_train.values.reshape(y_train.shape[0], )
    y_test = y_test.values.reshape(y_test.shape[0], )


    params = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 500),
        "max_depth": trial.suggest_int("max_depth", 2, 50),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", 1]),
        "ccp_alpha": trial.suggest_float("ccp_alpha", 1e-3, 10.0, log=True),
        'max_leaf_nodes': trial.suggest_int("max_leaf_nodes", 2, X_train.shape[0]),
        'min_samples_split': trial.suggest_int("min_samples_split", 2, 10)
    }

    kf = KFold(n_splits=N_SPLITS)

    regr = RandomForestRegressor(max_depth = params['max_depth'],
                                                        n_estimators = params['n_estimators'],
                                                        max_features = params['max_features'], 
                                                        ccp_alpha = params['ccp_alpha'],
                                                        max_leaf_nodes = params['max_leaf_nodes'],
                                                        min_samples_split = params['min_samples_split'],
                                                        criterion = 'squared_error',
                                                        random_state=0)    
    # regr.fit(X_train, y_train)


    # scores = cross_val_score(regr, X_train, y_train, scoring='r2', cv=kf, n_jobs=-1, verbose=0 )
    
    cv_results = cross_validate(regr, X_train, y_train, scoring=('r2'), cv=kf,
                                            n_jobs=-1, verbose=0, return_estimator=True)

    estimator = cv_results['estimator']
    test_scores = np.mean(cv_results['test_score'])

    # print(scores)

    return test_scores

In [41]:

def __objective(trial):

    X, y  = get_dataset()

    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", 1]),
        "ccp_alpha": trial.suggest_float("ccp_alpha", 1e-3, 1.0, log=True),
        'max_leaf_nodes': trial.suggest_int("max_leaf_nodes", 2, X.shape[0]),
        'min_samples_split': trial.suggest_int("min_samples_split", 2, 10)
    }

    kf = KFold(n_splits=N_SPLITS, )

    avg_r2_train_list = []
    avg_r2_test_list = []

    for train_index, test_index in kf.split(X):

        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]

        y_train = y_train.values.reshape(y_train.shape[0], )
        y_test = y_test.values.reshape(y_test.shape[0], )

        regr = RandomForestRegressor(max_depth = params['max_depth'],
                                                            n_estimators = params['n_estimators'],
                                                            max_features = params['max_features'], 
                                                            # ccp_alpha = params['ccp_alpha'],
                                                            max_leaf_nodes = params['max_leaf_nodes'],
                                                            min_samples_split = params['min_samples_split'],
                                                            criterion = 'squared_error',
                                                            random_state=0)    
        regr.fit(X_train, y_train)
        
        r2_train = regr.score(X_train, y_train)
        r2_test = regr.score(X_test, y_test)

        avg_r2_train_list.append(r2_train)
        avg_r2_test_list.append(r2_test)
        
    avg_train_score = np.mean(avg_r2_train_list)
    avg_test_score = np.mean(avg_r2_test_list)

    abs_diff = np.sqrt(1 - abs(avg_test_score - avg_train_score))
    
    final_score = np.round((avg_train_score * avg_test_score * abs_diff) / (avg_train_score + avg_test_score), 3)


    print(f'avg. train score: {avg_train_score}')
    print(f'avg. test score: {avg_test_score}')
    print(f'final score: {final_score}')


    # err = mean_squared_error(pred_regr, y_test)
    return final_score


In [42]:
import optuna
import dask_optuna
import joblib

# EPOCHS = 30

# with Client() as client:
client = dasker.get_dask_client()
print(f"Dask dashboard is available at {client.dashboard_link}")

storage = dask_optuna.DaskStorage()
study = optuna.create_study(storage=storage, direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())

with joblib.parallel_backend("dask"):
    study.optimize(objective, n_trials=100, n_jobs=-1)

print("Number of trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

best_trial = study.best_trial

print('Best Parameters')
for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))



[I 2022-10-05 09:09:57,825] A new study created in memory with name: no-name-d3a6d9a3-0f5e-428c-954f-d26750fdd6b0


Dask dashboard is available at http://127.0.0.1:8787/status


[I 2022-10-05 09:10:10,590] Trial 3 finished with value: 0.5252878262030252 and parameters: {'n_estimators': 35, 'max_depth': 36, 'max_features': 'sqrt', 'ccp_alpha': 0.7590519992204429, 'max_leaf_nodes': 95, 'min_samples_split': 4}. Best is trial 3 with value: 0.5252878262030252.
[I 2022-10-05 09:10:12,094] Trial 15 finished with value: 0.5120792150670553 and parameters: {'n_estimators': 44, 'max_depth': 48, 'max_features': 'log2', 'ccp_alpha': 0.8324743727598442, 'max_leaf_nodes': 100, 'min_samples_split': 6}. Best is trial 3 with value: 0.5252878262030252.
[I 2022-10-05 09:10:14,259] Trial 8 finished with value: 0.45433879840465813 and parameters: {'n_estimators': 42, 'max_depth': 17, 'max_features': 1, 'ccp_alpha': 0.07909980290375974, 'max_leaf_nodes': 89, 'min_samples_split': 7}. Best is trial 3 with value: 0.5252878262030252.
[I 2022-10-05 09:10:15,085] Trial 14 finished with value: 0.512294187609188 and parameters: {'n_estimators': 28, 'max_depth': 26, 'max_features': 'sqrt', '

Number of trials: 100
Best trial:
Best Parameters
n_estimators: 497
max_depth: 32
max_features: sqrt
ccp_alpha: 1.667107530460356
max_leaf_nodes: 21
min_samples_split: 3


[I 2022-10-05 09:17:05,279] Trial 98 finished with value: 0.5091102749195313 and parameters: {'n_estimators': 498, 'max_depth': 28, 'max_features': 'sqrt', 'ccp_alpha': 0.9671600081080614, 'max_leaf_nodes': 7, 'min_samples_split': 3}. Best is trial 76 with value: 0.5308043262479213.
[I 2022-10-05 09:17:05,321] Trial 99 finished with value: 0.509496090406148 and parameters: {'n_estimators': 500, 'max_depth': 29, 'max_features': 'sqrt', 'ccp_alpha': 0.6683063313132895, 'max_leaf_nodes': 7, 'min_samples_split': 3}. Best is trial 76 with value: 0.5308043262479213.
